In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

Helper functions and imports

In [ ]:
import json

import pandas as pd
import pickle

Create an index which references which smiles have which atom types and which spectra available.

In [ ]:
def read_json(path):
    with open(path, "r") as infile:
        dat = json.load(infile)
    return dat

In [ ]:
data = pickle.load(open("data/221205/xanes.pkl", "rb"))

In [ ]:
functional_group_data = read_json("data/221205/functional_groups.json")
all_functional_groups_enumerated = [g for groups in functional_group_data.values() for g in groups]

In [ ]:
all_unique_functional_groups = sorted(list(set(all_functional_groups_enumerated)))

In [ ]:
index = {
    "SMILES": [],
    "C": [],
    "N": [],
    "O": [],
    "C-XANES": [],
    "N-XANES": [],
    "O-XANES": []
}
index = {**index, **{fg: [] for fg in all_unique_functional_groups}}

In [ ]:
for smile, dat in data["data"].items():
    
    lower_smile = smile.lower()
    
    index["SMILES"].append(smile)
    
    for key in ["C", "N", "O"]:
        index[key].append(int(key.lower() in lower_smile))
    
    for key in ["C-XANES", "N-XANES", "O-XANES"]:
        index[key].append(int(dat[key] is not None))
        
    for fg in all_unique_functional_groups:
        index[fg].append(int(fg in functional_group_data[smile]))

In [ ]:
df = pd.DataFrame(index)

In [ ]:
df.to_csv("data/221205/index.csv")

In [ ]:
index = pd.read_csv("data/221205/index.csv", index_col=0)  # Reload

In [ ]:
assert (df == index).all().all()

Test the usage of the index.

In [ ]:
from multimodal_molecules.data import get_dataset

In [ ]:
data = get_dataset(
    xanes_path="data/221205/xanes.pkl",
    index_path="data/221205/index.csv",
    conditions="C-XANES,!N"
)